In [9]:
import openai
from pymongo import MongoClient
import chromadb
from chromadb.config import Settings
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
import os
import openai

from langgraph.graph import StateGraph, END
from langgraph.graph.message import MessageGraph
from typing import TypedDict, Annotated
from langchain_core.messages import HumanMessage
from langchain_core.runnables import Runnable, RunnableLambda

In [10]:
# Conexão com o ChromaClient Persistente
chroma_client = chromadb.PersistentClient(path="./chroma_db")



In [11]:
# Chave da API da openai
openai_api_key = os.getenv("OPENAI_API_KEY")

# Conectar ao MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["rag_db"]
mongo_chunks = mongo_db["chunks"]
image_chunks = mongo_db["image_chunks"]



collection = chroma_client.get_collection("rag_embeddings")
image_collection = chroma_client.get_collection("image_contexts")

# Inicializar embeddings
embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key) #text-embedding-ada-002

In [12]:
collection_images = chroma_client.get_or_create_collection("image_contexts")


In [13]:
query = "APA"
query_embedding = embedding_model.embed_query(query)
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5
)
print(results)
chunk_ids = [
    result["source_file"] + "_chunk_" + str(result["chunk_index"])
    for result in results["metadatas"][0]
]

chunks_text = []
for chunk_id in chunk_ids:
    chunk = mongo_chunks.find_one({"chunk_id": chunk_id})
    if chunk:
        chunks_text.append(chunk["chunk_text"])
        print("Contexto retirado do documento:")
        print(chunk["pdf_file"])
        print("Link para o documento:")
        print(chunk["source_url"])
print(chunks_text)

{'ids': [['EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_263', 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_270', 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_262', 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_275', 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_240']], 'embeddings': None, 'documents': [[None, None, None, None, None]], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'source_file': 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf', 'source_url': 'https://cetesb.sp.gov.br/eiarima/eia/EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf', 'chunk_index': 263}, {'source_file': 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf', 'chunk_index': 270, 'source_url

In [14]:


# Definindo os estados compartilhados
class ChatState(TypedDict):
    input: str
    query: str
    context: str
    image_context: str
    answer: str
    validation: str

# Agente de busca textual
def retrieve_text_context(state: ChatState) -> ChatState:
    query = state["input"]
    query_embedding = embedding_model.embed_query(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5
    )

    chunk_ids = [
        result["source_file"] + "_chunk_" + str(result["chunk_index"])
        for result in results["metadatas"][0]
    ]

    chunks_text = []
    for chunk_id in chunk_ids:
        chunk = mongo_chunks.find_one({"chunk_id": chunk_id})
        if chunk:
            chunks_text.append(chunk["chunk_text"])
            print("Contexto retirado do documento:")
            print(chunk["pdf_file"])
            print("Link para o documento:")
            print(chunk["source_url"])

    return {
        **state,
        "query": query,
        "context": "\n\n".join(chunks_text)
    }

# Agente de busca de contexto por imagens relacionadas à query
def retrieve_image_context(state: ChatState) -> ChatState:
    query = state["input"]
    query_embedding = embedding_model.embed_query(query)

    results = image_collection.query(
        query_embeddings=[query_embedding], 
        n_results=5
    )

    image_contexts = []

    for metadata in results["metadatas"][0]:
        chunk_id = metadata["source_file"] + "_chunk_" + str(metadata["chunk_index"])
        chunk = image_chunks.find_one({"chunk_id": chunk_id})
        if chunk:
            image_contexts.append(chunk["text"])

    return {
        **state,
        "image_context": "\n\n".join(image_contexts)
    }


# Agente que gera a resposta usando OpenAI
def generate_answer(state: ChatState) -> ChatState:
    client = openai.OpenAI(api_key=openai_api_key)

    prompt = f"""
Você é um assistente especialista em licenciamento ambiental, com foco em auxiliar na elaboração de documentos EIA e RIMA. Use exclusivamente os contextos a seguir para responder de forma precisa e técnica.

Contexto textual:
{state['context']}

Contexto de imagens (descrições ou legendas):
{state['image_context']}

Pergunta:
{state['query']}
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um assistente técnico ambiental especializado em EIA e RIMA."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=1000
    )

    return {
        **state,
        "answer": response.choices[0].message.content.strip()
    }

# Agente validador da resposta
def AnswerValidatorAgent(state: ChatState) -> ChatState:
    client = openai.OpenAI(api_key=openai_api_key)

    answer = state["answer"]
    context = state["context"]
    image_context = state["image_context"]
    query = state["query"]

    validation_prompt = f"""
Você é um validador técnico. Avalie a resposta de um assistente que tenta responder perguntas sobre EIA e RIMA com base em contextos textuais e visuais.

Contexto textual:
{context}

Contexto de imagens:
{image_context}

Pergunta feita:
{query}

Resposta do assistente:
{answer}

Agora avalie a resposta com base nos critérios abaixo:
1. A resposta está diretamente relacionada aos contextos fornecidos?
2. Há alguma afirmação vaga, genérica ou sem apoio no contexto?
3. A linguagem está tecnicamente correta e clara?

Se a resposta estiver satisfatória, diga "VALIDADO".
Se não, explique o problema encontrado.
"""

    validation_response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um validador técnico de respostas geradas por IA."},
            {"role": "user", "content": validation_prompt}
        ],
        temperature=0,
        max_tokens=500
    )

    validation = validation_response.choices[0].message.content.strip()

    return {
        **state,
        "validation": validation
    }

# Construção do grafo
builder = StateGraph(ChatState)

# Adiciona os nós
builder.add_node("text_retriever", retrieve_text_context)
builder.add_node("image_retriever", retrieve_image_context)
builder.add_node("chat", generate_answer)
builder.add_node("validator", AnswerValidatorAgent)

# Define conexões entre os nós
builder.set_entry_point("text_retriever")
builder.add_edge("text_retriever", "image_retriever")
builder.add_edge("image_retriever", "chat")
builder.add_edge("chat", "validator")
builder.add_edge("validator", END)

# Compila o grafo
graph = builder.compile()



In [15]:
# Iteração com o chat
while True:
    user_input = input("Você: ")
    if user_input.lower() in ["sair", "exit", "quit"]:
        break

    result = graph.invoke({"input": user_input})
    print("\nBot:", result["answer"])
    print("Validação:", result["validation"])


Você:  Quais municipios estão incluidos no APA?


Contexto retirado do documento:
EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Link para o documento:
https://cetesb.sp.gov.br/eiarima/eia/EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Contexto retirado do documento:
EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Link para o documento:
https://cetesb.sp.gov.br/eiarima/eia/EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Contexto retirado do documento:
EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Link para o documento:
https://cetesb.sp.gov.br/eiarima/eia/EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Contexto retirado do documento:
EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Link para o documento:
https://cetesb.sp.gov.br/eiarima/eia/EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Contexto retirad

Você:  sair
